In [ ]:
!pip install agentpy owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 20.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.9/778.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
  Created wheel for owlready2: filename=owlready2-0.46-cp310-cp310-linux_x86_64.whl size=24073560 sha256=0b56129a87293fa2a1791b7de804221f18d88a7c8f67210349fefaab43491445
  Stored in directory: /root/.cache/pip/wheels/bf/6f/ef/35df148baa59fc6b243aca4f101e17aed14b67dc1166b73fe0
Successfully built owlready2


In [ ]:
import agentpy as ap
from owlready2 import *
from matplotlib import pyplot as plt
import IPython
import random
import json

In [ ]:
onto = get_ontology("file://onto.owl")

In [ ]:
# Global Variables
warningPoint = (8, 10)
isWarningAlert = True
droneInWarningPos = False

In [ ]:
with onto:
    class Entity(Thing):
      pass

    class Drone(Thing):
      pass

    class Guard(Thing):
      pass

    class Camera(Thing):
      pass

    class Wall(Thing):
      pass

    class World(Thing):
      pass

    class Place(Thing):
      pass

    class has_position(DataProperty, FunctionalProperty):
      domain = [Entity]
      range = [str]

    class has_place(ObjectProperty, FunctionalProperty):
      domain = [Entity]
      range = [Place]

In [ ]:
class DroneAgent(ap.Agent):
  def see(self, grid):
    pass

  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()
          return

  def setup(self):
    self.agentType = 0
    self.direction = (-1, 0)
    self.flagPoints = [(10,2), (2,2), (2,6), (13,6), (13,10), (2,10), (2,14), (13,14), (13,18), (2,18)]
    self.isReturn = False
    self.isWarning = isWarningAlert
    self.warningPoint = warningPoint
    self.inWarningPos = droneInWarningPos
    self.droneReturned = False
    self.rules = (self.rule_1, self.rule_2, self.rule_3, self.rule_4, self.rule_5, self.rule_6, self.rule_7)
    self.actions = (self.returnDrone, self.landDrone, self.forward, self.moveUp, self.moveDown, self.moveLeft, self.moveRight)

  def step(self):
    self.see(self.model.grid)
    self.next()

  #RULES
  def rule_1(self, act): # Forward
    validador = [False, False]
    isFlag = False
    pos = self.model.grid.positions[self]
    if not self.inWarningPos:
      for flag in self.flagPoints:
        if pos == flag:
          isFlag = True
      if not isFlag:
        validador[0] = True
    if act == self.forward:
        validador[1] = True
    return sum(validador) == 2

  def rule_2(self, act): # MoveUp
    validador = [False, False]
    pos = self.model.grid.positions[self]
    if pos == self.flagPoints[0]:
      self.droneReturned = False
      self.isReturn = False
      validador[0] = True
    if not self.inWarningPos:
      for i in range(0, len(self.flagPoints)):
        if i % 4 == 0 and not self.isReturn:
          if pos == self.flagPoints[i]:
            validador[0] = True
        elif i % 4 == 3 and self.isReturn:
          if pos == self.flagPoints[i]:
            validador[0] = True
    if act == self.moveUp:
        validador[1] = True
    return sum(validador) == 2

  def rule_3(self, act): # MoveDown
    validador = [False, False]
    pos = self.model.grid.positions[self]
    if pos == self.flagPoints[len(self.flagPoints)-1]:
      self.isReturn = True
      validador[0] = True
    if not self.inWarningPos:
      for i in range(0, len(self.flagPoints)):
        if i % 4 == 2 and not self.isReturn:
          if pos == self.flagPoints[i]:
            validador[0] = True
        elif i % 4 == 1 and self.isReturn:
          if pos == self.flagPoints[i]:
            validador[0] = True
    if act == self.moveDown:
        validador[1] = True
    return sum(validador) == 2

  def rule_4(self, act): # MoveLeft
    validador = [False, False]
    pos = self.model.grid.positions[self]
    if self.isReturn and not self.inWarningPos:
      for i in range(0, len(self.flagPoints)):
        if i % 2 == 0 and i != 0:
          if pos == self.flagPoints[i]:
            validador[0] = True
    if act == self.moveLeft:
        validador[1] = True
    return sum(validador) == 2

  def rule_5(self, act): # MoveRight
    validador = [False, False]
    pos = self.model.grid.positions[self]
    if not self.isReturn and not self.inWarningPos:
      for i in range(0, len(self.flagPoints)):
        if i % 2 == 1 and i != 9:
          if pos == self.flagPoints[i]:
            validador[0] = True
    if act == self.moveRight:
        validador[1] = True
    return sum(validador) == 2

  def rule_6(self, act): # LandDrone
    validador = [False, False]
    pos = self.model.grid.positions[self]
    if self.isWarning:
      if pos == self.warningPoint:
        validador[0] = True
    if act == self.landDrone:
        validador[1] = True
    return sum(validador) == 2

  def rule_7(self, act): # ReturnDrone
    validador = [False, False]
    pos = self.model.grid.positions[self]
    if self.isWarning and not self.droneReturned:
      if warningPoint[1] < pos[1]:
        validador[0] = True
      elif warningPoint[1] == pos[1] and warningPoint[0] > pos[0]:
        validador[0] = True
    if act == self.returnDrone:
        validador[1] = True
    return sum(validador) == 2

  #ACTIONS
  def returnDrone(self):
    self.droneReturned = True
    self.isReturn = True
    self.direction = (self.direction[0] * -1, self.direction[1] * -1)

  def landDrone(self):
    droneInWarningPos = True

  def moveUp(self):
    self.direction = (-1, 0)
    self.forward()

  def moveDown(self):
    self.direction = (1, 0)
    self.forward()

  def moveLeft(self):
    self.direction = (0, -1)
    self.forward()

  def moveRight(self):
    self.direction = (0, 1)
    self.forward()

  def forward(self):
    self.model.grid.move_by(self, self.direction)

In [ ]:
class GuardAgent(ap.Agent):
  def setup(self):
    self.agentType = 1
    self.first_step = True
    self.pos = None
    self.isWarning = isWarningAlert
    self.inWarningPos = droneInWarningPos
    self.rules = (self.rule_1, self.rule_2, self.rule_3)
    self.actions = (self.idle, self.alert, self.controll)

  def step(self):
    if self.first_step:
      self.pos = self.model.grid.positions[self]
      self.first_step = False
    self.next()

  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()
          return

  #RULES
  def rule_1(self, act): # Idle
    validador = [False, False]
    if not self.isWarning:
      if not self.inWarningPos:
        validador[0] = True
    if act == self.idle:
        validador[1] = True
    return sum(validador) == 2

  def rule_2(self, act): # Alert
    validador = [False, False]
    if self.isWarning:
      if not self.inWarningPos:
        validador[0] = True
    if act == self.alert:
        validador[1] = True
    return sum(validador) == 2

  def rule_3(self, act): # Controll
    validador = [False, False]
    if self.isWarning:
      if self.inWarningPos:
        validador[0] = True
    if act == self.controll:
        validador[1] = True
    return sum(validador) == 2

  #ACTIONS
  def idle(self):
    pass

  def alert(self):
    pass

  def controll(self):
    pass

In [ ]:
class CameraAgent(ap.Agent):
  def setup(self):
    self.agentType = 2
    self.first_step = True
    self.pos = None
    self.isWarning = isWarningAlert
    self.rules = (self.rule_1, self.rule_2)
    self.actions = (self.notify, self.idle)

  def step(self):
    if self.first_step:
      self.pos = self.model.grid.positions[self]
      self.first_step = False
    self.next()

  def next(self):
    for act in self.actions:
      for rule in self.rules:
        if rule(act):
          act()
          return

  #RULES
  def rule_1(self, act): # Idle
    validador = [False, False]
    if not self.isWarning:
      validador[0] = True
    if act == self.idle:
        validador[1] = True
    return sum(validador) == 2

  def rule_2(self, act): # Notify
    validador = [False, False]
    if self.isWarning:
      validador[0] = True
    if act == self.notify:
        validador[1] = True
    return sum(validador) == 2

  #ACTIONS
  def idle(self):
    pass

  def notify(self):
    pass

In [ ]:
class WallAgent(ap.Agent):
  def setup(self):
    self.agentType = 3
    self.first_step = True
    self.pos = None

  def step(self):
    if self.first_step:
      self.pos = self.model.grid.positions[self]
      self.first_step = False

In [ ]:
class WarehouseModel(ap.Model):
    def setup(self):
        self.drones = ap.AgentList(self, len(self.p.drones), DroneAgent)
        self.guards = ap.AgentList(self, len(self.p.guards), GuardAgent)
        self.cameras = ap.AgentList(self, len(self.p.cameras), CameraAgent)
        self.walls = ap.AgentList(self, len(self.p.walls), WallAgent)
        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)
        self.grid.add_agents(self.drones, positions=self.p.drones, empty=True)
        self.grid.add_agents(self.guards, positions=self.p.guards, empty=True)
        self.grid.add_agents(self.cameras, positions=self.p.cameras, empty=True)
        self.grid.add_agents(self.walls, positions=self.p.walls, empty=True)

    def step(self):
        self.drones.step()
        self.guards.step()
        self.cameras.step()
        self.walls.step()

In [ ]:
def animation_plot(model, ax):
    agent_type_grid = model.grid.attr_grid('agentType')
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Warehouse Model \n Time-step: {model.t}, ")

request_data = '''{
   "warehouse": [
    ["W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W"],
    ["W", "C", " ", " ", " ", " ", " ", " ", "W", "C", " ", " ", " ", " ", " ", " ", "W", "C", " ", " ", "W"],
    ["W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", "D", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", "R", " ", "W"],
    ["W", "W", " ", "W", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W"],
    ["W", " ", "G", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W", " ", " ", " ", " ", " ", " ", " ", "W"],
    ["W", " ", " ", " ", "W", "C", " ", " ", " ", " ", " ", " ", "W", "C", " ", " ", " ", " ", " ", " ", "W"],
    ["W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W", "W"]
  ]
}'''

# Parse the JSON data
data = json.loads(request_data)
warehouse = data['warehouse']

parameters = {
    'M': len(warehouse),
    'N': len(warehouse[0]),
    'steps': 50,
    'drones': [],
    'guards': [],
    'walls': [],
    'cameras': [],
}

# Process the matrix
for row_idx, row in enumerate(warehouse):
    for col_idx, cell in enumerate(row):
        if cell == 'W':
            parameters['walls'].append((row_idx, col_idx))
        elif cell == 'G':
            parameters['guards'].append((row_idx, col_idx))
        elif cell == 'D':
            parameters['drones'].append((row_idx, col_idx))
        elif cell == 'C':
            parameters['cameras'].append((row_idx, col_idx))

In [ ]:
fig, ax = plt.subplots()
model = WarehouseModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml())